In [1]:
suppressMessages(library(lubridate))
suppressMessages(library(dplyr))
suppressMessages(library(data.table))
suppressMessages(library(gsheet))
suppressMessages(library(ggplot2))
suppressMessages(library(EnvStats))
suppressMessages(library(tidyverse))

Warning message:
“package ‘gsheet’ was built under R version 4.2.2”
Warning message:
“package ‘EnvStats’ was built under R version 4.2.2”


In [2]:
setwd("../../")
source("/workdir/omm35/paper_urine_cfrna/scripts/theme_ggplot_cfrna.R")

In [3]:
rm(list=ls())
# Load QC-passed samples metadata with only 'sample_id' and 'biofluid' columns
samples_after_qc <- fread("/workdir/omm35/paper_urine_cfrna/metadata/QC_pass_samples.csv")
meta.df = samples_after_qc[, c("sample_id", "biofluid")]

# Set working directory to where cfRNA length profiles are stored
setwd("/local/workdir/omm35/cfRNA_length_profiles")

# Define a function to process each file: read counts, clean sample_id, and calculate fraction
get_length <- function(filename) {
  df <- read.table(filename, header = FALSE, col.names = c('count', 'fragment_length'))
  df$sample_id = filename
  df$sample_id = gsub(".*/", "", df$sample_id) # Remove path, leaving filename
  df$sample_id = gsub("_aligned.lengths.counts", "", df$sample_id) # Clean sample_id
  df$frac <- df$count / sum(df$count) # Calculate fraction of counts for each fragment length
  return(df)
}

# List all files matching the pattern "*.lengths.counts" in a specified directory, including subdirectories
all.files <- list.files("/local/workdir/omm35/cfRNA_length_profiles/output/", full.names = TRUE, pattern = "*.lengths.counts", recursive = TRUE)

# Apply the 'get_length' function to each file and combine the results into one data frame
all.length.df = rbindlist(lapply(all.files, FUN = get_length))

# Filter to include only samples that passed QC
all.length.df = all.length.df[all.length.df$sample_id %in% meta.df$sample_id, ]

# Merge cfRNA length data with sample metadata
all.length.df = meta.df %>% left_join(all.length.df, by = "sample_id")

# Prepare the combined data frame for plotting
combined.plasma.urine.df = all.length.df

# Set options for plot size
options(repr.plot.width = 10, repr.plot.height = 10)

# Define a colorblind-friendly color palette for plotting
safe_colorblind_palette <- c("#88CCEE", "#CC6677", "#DDCC77", "#117733", "#332288", "#AA4499", 
                             "#44AA99", "#999933", "#882255", "#661100", "#6699CC", "#888888")
safe_colorblind_palette <- c('#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE', '#AA3377', '#BBBBBB')

# Plot cfRNA fragment length distributions by biofluid using ggplot2
ggplot(combined.plasma.urine.df, aes(y = frac, x = fragment_length, group = sample_id, color = biofluid)) +
  geom_line(alpha = 0.001) +  # Plot individual samples with very low opacity
  stat_summary(aes(group = biofluid), fun = mean, geom = "line", size = 0.2) +  # Overlay average lines per biofluid
  theme_classic() +  # Use classic theme
  xlim(0, max(combined.plasma.urine.df$fragment_length) - 450) +  # Set x-axis limits
  ylim(0, 0.0095) +  # Set y-axis limits
  xlab('Fragment length (bp)') +  # Label x-axis
  ylab('Fraction of reads') +  # Label y-axis
  theme(  # Customize various theme elements for readability and aesthetics
    axis.title.x = element_text(size = 8, family = "Helvetica", color = "black"),
    axis.text.x = element_text(size = 8, family = "Helvetica", color = "black"),
    axis.title.y = element_text(size = 8, family = "Helvetica", color = "black"),
    axis.text.y = element_text(size = 8, family = "Helvetica", color = "black"),
    legend.position = "right",
    legend.text = element_text(size = 6),
    legend.title = element_blank(),
    legend.box.background = element_rect()
  ) +
  scale_colour_manual(values = safe_colorblind_palette)  # Apply custom color palette

In [8]:
# length(unique(all.length.df$sample_id))
# print("")
# length(unique(samples_after_qc$sample_id))
# head(all.length.df)
# head(meta.df)

[1] 315

[1] ""


[1] 315

calculate stats

In [2]:
# head(combined.plasma.urine.df) 
# #SD = sqrt(sum((df$x−Mean)**2*df$frequency)/(sum(df$frequency)−1))
# #combined.plasma.urine.df  %>% group_by(biofluid) %>% summarize(mean_length = mean(rep(fragment_length,count)))
# d = combined.plasma.urine.df  %>% group_by(biofluid,sample_id) %>% summarize(mean_length = sum(fragment_length*count)/sum(count))
# d %>% ggplot(.,aes(biofluid,mean_length, fill=biofluid))+geom_boxplot()+
# geom_jitter(width = 0.15)+stat_n_text()+ theme_classic()+
# theme(
# #legend.position = c(0.8, 0.9),
# #legend.position="top",
# legend.position="none",
# legend.title = element_blank(),
# axis.title.x = element_blank(),
# axis.text.x = element_text(size = 10,family = "Helvetica",color="black"),
# axis.title.y = element_text(size = 10,family = "Helvetica", color="black"),
# axis.text.y = element_text(size=10,family = "Helvetica", color="black"),  
# ) + ylab("Mean Length")  + scale_fill_manual(values=safe_colorblind_palette)

In [27]:
df = combined.plasma.urine.df  %>% group_by(biofluid,sample_id) %>% filter(count==max(count))
df = df  %>% group_by(biofluid) %>% summarize(mean=mean(fragment_length),sdev=sd(fragment_length))
df
df = df  %>% filter((biofluid != "ktx_urine")) 
df

biofluid,mean,sdev
<chr>,<dbl>,<dbl>
aki_plasma,114.88235,34.16783
aki_urine,123.08824,27.05143
gvhd_plasma,96.53608,23.29792
gvhd_urine,102.15464,46.84379
healthy_plasma,134.20000,18.24007
healthy_urine,99.80000,34.05437
ktx_urine,105.81395,67.41423


biofluid,mean,sdev
<chr>,<dbl>,<dbl>
aki_plasma,114.88235,34.16783
aki_urine,123.08824,27.05143
gvhd_plasma,96.53608,23.29792
gvhd_urine,102.15464,46.84379
healthy_plasma,134.20000,18.24007
healthy_urine,99.80000,34.05437


In [40]:
df = combined.plasma.urine.df  %>% group_by(biofluid,sample_id)
df = df  %>% filter((biofluid != "ktx_urine")) 
df = df  %>% group_by(biofluid,sample_id) %>% summarise(weighted_mean = weighted.mean(x = fragment_length, w = count))
df$fluid=gsub("healthy_|gvhd_|aki_","",df$biofluid)
df  %>% group_by(fluid) %>% summarize(mean=mean(weighted_mean),sdev=sd(weighted_mean))

`summarise()` has grouped output by 'biofluid'. You can override using the `.groups` argument.


fluid,mean,sdev
<chr>,<dbl>,<dbl>
plasma,146.0138,24.69854
urine,151.9999,29.91541
